# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
import tensorflow as tf
import keras

import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.utils import np_utils
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.constraints import maxnorm

Using TensorFlow backend.


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
from keras.datasets import cifar10

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
X_train_04 = np.asarray([X_train[key] for (key, label) in enumerate(y_train) if int(label) >= 0 and int(label) <= 4])
X_train_59 = np.asarray([X_train[key] for (key, label) in enumerate(y_train) if int(label) >= 5 and int(label) <= 9])

X_test_04 = np.asarray([X_test[key] for (key, label) in enumerate(y_test) if int(label) >= 0 and int(label) <= 4])
X_test_59 = np.asarray([X_test[key] for (key, label) in enumerate(y_test) if int(label) >= 5 and int(label) <= 9])

In [0]:
y_train_04 = np.asarray([y_train[key] for (key, label) in enumerate(y_train) if int(label) >= 0 and int(label) <= 4])
y_train_59 = np.asarray([y_train[key] for (key, label) in enumerate(y_train) if int(label) >= 5 and int(label) <= 9])

y_test_04 = np.asarray([y_test[key] for (key, label) in enumerate(y_test) if int(label) >= 0 and int(label) <= 4])
y_test_59 = np.asarray([y_test[key] for (key, label) in enumerate(y_test) if int(label) >= 5 and int(label) <= 9])

In [0]:
np.unique(y_test_04)

array([0, 1, 2, 3, 4])

In [0]:
np.unique(y_test_59)

array([5, 6, 7, 8, 9])

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
num_class_04 = len(np.unique(y_train_04))

y_train_04 = tf.keras.utils.to_categorical(y_train_04, num_classes=num_class_04)
y_test_04 = tf.keras.utils.to_categorical(y_test_04, num_classes=num_class_04)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
# normalize inputs from 0-255 to 0.0-1.0
X_train_04 = X_train_04.astype('float32')
X_test_04 = X_test_04.astype('float32')

X_train_04 = X_train_04 / 255.0
X_test_04 = X_test_04 / 255.0

In [0]:
# initialize model
model = Sequential()

# add input layer with input_size(32,32,3)
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))

# create Conv 2D layers , along with DropoUts and MaxPooling to avoid overfit
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
# Flatten output
model.add(Flatten())
model.add(Dropout(0.2))
# add Dense layers
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

# add Output layer
model.add(Dense(num_class_04, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Compile model
from keras.optimizers import SGD
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = tf.keras.optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

In [0]:
# Fit the model
model.fit(X_train_04, y_train_04, validation_data=(X_test_04, y_test_04), epochs=epochs, batch_size=64)


Train on 25000 samples, validate on 5000 samples
Epoch 1/25
25000/25000 [==============================] - 217s 9ms/sample - loss: 1.4088 - acc: 0.3918 - val_loss: 1.3041 - val_acc: 0.4416
Epoch 2/25
25000/25000 [==============================] - 215s 9ms/sample - loss: 1.0857 - acc: 0.5528 - val_loss: 1.0236 - val_acc: 0.5904
Epoch 3/25
25000/25000 [==============================] - 214s 9ms/sample - loss: 0.9295 - acc: 0.6293 - val_loss: 0.8545 - val_acc: 0.6754
Epoch 4/25
25000/25000 [==============================] - 214s 9ms/sample - loss: 0.8358 - acc: 0.6701 - val_loss: 0.8108 - val_acc: 0.6870
Epoch 5/25
25000/25000 [==============================] - 215s 9ms/sample - loss: 0.7724 - acc: 0.7007 - val_loss: 0.8270 - val_acc: 0.6774
Epoch 6/25
25000/25000 [==============================] - 214s 9ms/sample - loss: 0.7138 - acc: 0.7250 - val_loss: 0.6858 - val_acc: 0.7402
Epoch 7/25
25000/25000 [==============================] - 216s 9ms/sample - loss: 0.6757 - acc: 0.7385 - val_lo

In [0]:
score = model.evaluate(X_test_04, y_test_04)
print('\n', 'Test accuracy :', score[1]*100)

5000/5000 [==============================] - 10s 2ms/sample - loss: 0.4803 - acc: 0.8258

 Test accuracy for Model : 82.58000016212463


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
#Freeze pre-trained model layers 
for layer in model.layers[:-5]:
    layer.trainable = False

In [0]:
for layer in model.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe715260e10> False
<tensorflow.python.keras.layers.core.Dropout object at 0x7fe7152640f0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe7152649b0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe715264898> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe71483b8d0> False
<tensorflow.python.keras.layers.core.Dropout object at 0x7fe715260fd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe714874f28> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe71483b940> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe7149913c8> False
<tensorflow.python.keras.layers.core.Dropout object at 0x7fe7149916a0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe71494ccf8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe7149b1320> False
<ten

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
# pre process labels
y_train_59 = y_train_59 - 5
y_test_59 = y_test_59 - 5

In [0]:
# one hot encoding
num_class_59 = len(np.unique(y_train_59))

y_train_59 = tf.keras.utils.to_categorical(y_train_59, num_classes=num_class_59)
y_test_59 = tf.keras.utils.to_categorical(y_test_59, num_classes=num_class_59)

In [0]:
# normalizze input
X_train_59 = X_train_59.astype("float32")/255.0
X_test_59 = X_test_59.astype("float32")/255.0

In [0]:
# fitting the model by retraining just dense layers 

epochs=15
batch_size = 64

model.fit(X_train_59, y_train_59, validation_data=(X_test_59, y_test_59), batch_size=batch_size, epochs=epochs)

Train on 25000 samples, validate on 5000 samples
Epoch 1/15
25000/25000 [==============================] - 218s 9ms/sample - loss: 0.9467 - acc: 0.6458 - val_loss: 0.5996 - val_acc: 0.7762
Epoch 2/15
25000/25000 [==============================] - 218s 9ms/sample - loss: 0.6038 - acc: 0.7751 - val_loss: 0.4940 - val_acc: 0.8192
Epoch 3/15
25000/25000 [==============================] - 218s 9ms/sample - loss: 0.5127 - acc: 0.8117 - val_loss: 0.4530 - val_acc: 0.8352
Epoch 4/15
25000/25000 [==============================] - 218s 9ms/sample - loss: 0.4685 - acc: 0.8256 - val_loss: 0.4262 - val_acc: 0.8434
Epoch 5/15
25000/25000 [==============================] - 217s 9ms/sample - loss: 0.4345 - acc: 0.8386 - val_loss: 0.4162 - val_acc: 0.8462
Epoch 6/15
25000/25000 [==============================] - 216s 9ms/sample - loss: 0.4023 - acc: 0.8520 - val_loss: 0.3987 - val_acc: 0.8598
Epoch 7/15
25000/25000 [==============================] - 215s 9ms/sample - loss: 0.3720 - acc: 0.8644 - val_lo

In [0]:
score = model.evaluate(X_test_59, y_test_59)
print('\n', 'Test accuracy with Transfer Learning :', score[1]*100)

5000/5000 [==============================] - 4s 709us/sample - loss: 0.5893 - accuracy: 0.7834

 Test accuracy for Model with Transfer Learning : 78.33999991416931


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
# use TfIDFVectorizer to create document-term matrices 
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()

In [0]:
twenty_train_dtm = vect.fit_transform(twenty_train.data)

In [0]:
twenty_test_dtm = vect.transform(twenty_test.data)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [0]:
# train the model using X_train_dtm
logreg.fit(twenty_train_dtm, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# class predictions for X_test_dtm
y_pred_class_log = logreg.predict(twenty_test_dtm)

In [0]:
# calculateing accuracy
from sklearn import metrics

metrics.accuracy_score(twenty_test.target, y_pred_class_log)

0.8868175765645806